In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import torch
from tqdm.notebook import trange
from tradezoo import (
    Account,
    Actor,
    Agent,
    Critic,
    Game,
    GeometricBrownianMotion,
    Market,
    MarketMaker,
    Trader,
    Trainer,
)

<IPython.core.display.Javascript object>

In [3]:
def make_agent():
    actor = Actor()
    critic = Critic()
    return Agent(
        actor=actor,
        actor_optimizer=torch.optim.Adam(actor.parameters(), lr=1e-4),
        critic=critic,
        critic_optimizer=torch.optim.Adam(critic.parameters(), lr=1e-3),
        discount_factor=0.99,
    )


num_traders = 4
traders = [
    Trader(
        agent=make_agent(),
        account=Account(cash_balance=256, stock_balance=256),
        market_maker=MarketMaker.inexhaustible(
            ask_process=GeometricBrownianMotion.driftless(
                initial_value=2,
                underlying_std=0.01,
            ),
            bid_process=GeometricBrownianMotion.driftless(
                initial_value=0.5,
                underlying_std=0.01,
            ),
        ),
    )
    for _ in range(num_traders)
]
game = Game.new(
    market=Market.from_accounts([trader.account for trader in traders]),
    traders=traders,
)
trainer = Trainer.new(game=game, replay_buffer_capacity=64)

<IPython.core.display.Javascript object>

In [4]:
batch_size = 16
for step_id in trange(1024):
    turn_result = trainer.turn_()
    if len(trainer.replay_buffers[turn_result.trader.agent].experiences) >= batch_size:
        trainer.train_step_(agent=turn_result.trader.agent, batch_size=batch_size)

  0%|          | 0/1024 [00:00<?, ?it/s]

ZeroDivisionError: float division by zero

<IPython.core.display.Javascript object>